# Kernel Density Estimation for Mouse Cell Detection Data   

**Using kernel density estimation (KDE) to analyze 2d cell detection data from flourescent imaging of stellate ganglia in mice.**

By Jihyun Park (`jihyunp@ics.uci.edu`) and Padhraic Smyth (`smyth@ics.uci.edu`)<br/>
Department of Computer Science, University of California, Irvine

Presented as part of the UCI BigDIPA Lab

September 2017

## Outline
-------------------------
In this section of the Lab we will use the kernel density estimation functions to analyze data from cell detections from flourescent imaging of stellate ganglia (near the heart) from mice.

This is unpublished data provided courtesy of Pradeep Rajendran and the UCLA Cardiac Arrhythmia Center, under funding from NIH SPARC: Mapping the mammalian cardiac nervous system.

 

## Requirements
--------------------------------
- Familiarity with the main KDE python notebook for the BigDIPA course
- Basic knowledge of probability.
- Familiarity with programming and nD array computing (e.g. working with matrices in Matlab or numpy in python).
- Python 3.5 with libraries : Jupyter (for ipython notebook), numpy, scipy, scikit-learn, matplotlib.
- It is recommended to have the newest version of libraries installed.

## References
----------------------------------
- [Scikit-learn Density Estimation](http://scikit-learn.org/stable/modules/density.html) : Description and examples on density estimation including KDE.
- [Scikit-learn KDE Package Documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html) : `KernelDensity` class documentation.

## Import Packages
-----------------------------

Import the necessary packages (you may have already imported them).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('classic')
%matplotlib inline

from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
# from sklearn.grid_search import GridSearchCV # if you have older version of sklearn

If you get an **`ImportError`** for **`model_selection`**, 
you can try importing **`GridSearchCV`** from **`sklearn.grid_search`** package.

- `from sklearn.grid_search import GridSearchCV`

or, update your **`scikit-learn`** using one of these commands below in **terminal window**.

- `conda install scikit-learn` : If you installed python through anaconda
- `pip install -U scikit-learn`




--------------------
##  Mouse Data

In [ ]:
# define the folder where data is located
datapath = './spatial_data/'

We will import 3 files below, where each file contains information about cell detections for 3 different mice. Each file contains locations of cell detections obtained via flourescent imaging of stellate ganglia (near the heart) of individual mice.

The format of each file is
- one row per detected cell
- first 3 columns are the x, y, z locations of the cells (here we will ignore the z dimension)
- the 4th column is an integer indicating the region of interest (ROI) that the cell is located in. We will focus on ROI 2, i.e., cells that have a 2 in column 4 (this is the ROI where most of the cells are located for each mouse).

The x,y locations of the cells for each mouse have been warped to try to register them, i.e., line up the data from each mouse as best as possible. This warping is not perfect, however, so you may see systematic shifts in the resulting data. The warping also means that the resulting x,y locations are not in any interpretable units

Thanks to Charless Fowlkes for preprocessing the data (warping, etc).

In [ ]:
# Read in the mouse data
filename = datapath + "mouse1_SG_L_W_data.csv"
m_data1_all = pd.read_csv(filename).as_matrix()
filename = datapath + "mouse2_SG_L_W_data.csv"
m_data2_all = pd.read_csv(filename).as_matrix()
filename = datapath + "mouse5_SG_L_W_data.csv"
m_data5_all = pd.read_csv(filename).as_matrix() 

# Print the shape of the data. We have 4 columns
print(m_data1_all.shape)
print(m_data2_all.shape)
print(m_data5_all.shape)

In [ ]:
m_data1_all[:10]

### Take the second region (ROI=2)

In [ ]:
r2_idx = np.where(m_data1_all[:,-1] == 2)[0]
m_data1 = m_data1_all[r2_idx, :2]

m_data1.shape

In [ ]:
# Do the same thing for data2 and data5

m_data2 = m_data2_all[np.where(m_data2_all[:,-1]==2)[0], :2]
print(m_data2.shape)
m_data5 = m_data5_all[np.where(m_data5_all[:,-1]==2)[0], :2]
print(m_data5.shape)

## 1. Scatter plot of the data

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
# Plot the data points
ax.scatter(m_data1[:,0], m_data1[:,1], s=10, color='blue', alpha=0.7, linewidth=0, label='Mouse1')
ax.scatter(m_data2[:,0], m_data2[:,1], s=10, color='red', alpha=0.7, linewidth=0, label='Mouse2')
ax.scatter(m_data5[:,0], m_data5[:,1], s=10, color='green', alpha=0.7, linewidth=0, label='Mouse5')
ax.grid(alpha=0.3)
ax.legend(loc='upper left')
plt.show()

### Three different plots

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(15,4), sharex=True, sharey=True)

axs[0].scatter(m_data1[:,0], m_data1[:,1], s=10, color='blue', alpha=0.7, linewidth=0, label='Mouse1')
axs[0].set_ylabel('Y')
axs[0].set_xlabel('X')
axs[0].set_title('Mouse 1')

axs[1].scatter(m_data2[:,0], m_data2[:,1], s=10, color='red', alpha=0.7, linewidth=0, label='Mouse2')
axs[1].set_xlabel('X')
axs[1].set_title('Mouse 2')

axs[2].scatter(m_data5[:,0], m_data5[:,1], s=10, color='green', alpha=0.7, linewidth=0, label='Mouse5')
axs[2].set_xlabel('X')
axs[2].set_title('Mouse 5')

plt.show()

### Grid arrays
Before we plot the density, we will generate arrays for grids. This is because we want a density value for each grid location. We're going to generate 100 X 100 mesh grid. If you want to make the grid denser, change the number for **`ngrid`** in the below code to something larger.

After generating the mesh grid, we're going to flatten the matrix to have (N x 2) shape so that it can be used as an input for other functions. <br\> 
The variables **`X, Y, xy, `**and **`ngrid`**  will be used throughout the lab. 

In [ ]:
# Check the minimum and maximum values of one of the data
print(np.min(m_data1[:,0]), np.max(m_data1[:,0]))
print(np.min(m_data1[:,1]), np.max(m_data1[:,1]))

In [ ]:
# Generate Mesh Grid for Plotting (ngrid x ngrid matrix)

# limits and grids for the data
lower_lim = 500
upper_lim = 1500

ngrid = 500

xgrid = np.linspace(lower_lim, upper_lim, ngrid)
ygrid = np.linspace(lower_lim, upper_lim, ngrid)

X, Y = np.meshgrid(xgrid, ygrid) # Now we have (ngrid x ngrid) matrix

# ravel() function flattens (ngrid x ngrid) matrix -> (1 x ngrid**2) array
xy = np.vstack([X.ravel(), Y.ravel()]).T

# Print the shapes!
print('Shape of X and Y: ' + str(X.shape))
print('Shape of xy: ' + str(xy.shape))

## 2. Define a wrapper function that runs KDE

Now define a wrapper function that runs KDE and returns the evaluated density $\hat{p}(x)=  \frac{1}{N} \sum_{i=1}^{N} K\left(x - x_i; h \right)$ in `(ngrid X ngrid)` matrix form. <br/>
It is better to define a wrapper function since this will be used a lot!

In [ ]:
def run_kde(Xdata, bandwidth, metric, kernel):
    """ Construct a KernelDensity object, fit with the data points we generated, 
        and then return the evaluated density for the (ngrid X ngrid) mesh grid """
    # Construct a kernel density object
    kde = KernelDensity(bandwidth=bandwidth, metric=metric, kernel=kernel)
    kde.fit(Xdata)
    # kde.score_samples() returns values in log scale
    # xy is the flattened mesh grid that we defined earlier (used as global var.)
    log_p_hat = kde.score_samples(xy)
    phat = np.exp(log_p_hat)
    phat = phat.reshape((ngrid, ngrid))
    return phat

print('Function run_kde() defined.')

## 3. Plot the estimated density

Now run the function defined above, and plot the estimated density $\hat{p}(x)$ as contour plot. 
We're going to use Euclidean distance (`metric='euclidean`) and Gaussian kernel (`kernel='gaussian'`), and change the size of the bandwidth to see the difference in the result. <br/>
(You can also try with different metrics or kernels : more information at [Scikit-learn KernelDensity](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html))

### 3.1 Manually Selected Bandwidth

Try running with different size of bandwidths to find the best bandwidth. Modify the variable **`selected_bw`** in the below code.

In [ ]:
# Generate the estimated density using kernel density estimation
selected_bw = 80

phat = run_kde(m_data1, bandwidth=selected_bw, metric='euclidean', kernel='gaussian') 

fig, axs = plt.subplots(1, 2, figsize=(11,5), sharex=True, sharey=True) 
ax = axs[0]
levels = np.linspace(phat.min(), phat.max(), 20)
im = ax.contourf(X, Y, phat, levels=levels, cmap='Blues')
ax.set_xlim(lower_lim, upper_lim)
ax.set_ylim(lower_lim, upper_lim)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Density from KDE (BW=%.2f)' % selected_bw)

ax = axs[1]
ax.scatter(m_data1[:,0], m_data1[:,1], s=35, c='blue', linewidth=0, alpha=0.6)
ax.set_xlabel('X')
plt.show()

In [ ]:
# First make it into a function
def run_kde_and_plot(data, bw, color='blue', cmap='Blues'):
    # Generate the estimated density using kernel density estimation
    phat = run_kde(data, bandwidth=bw, metric='euclidean', kernel='gaussian') 

    fig, axs = plt.subplots(1, 2, figsize=(11,5), sharex=True, sharey=True) 
    ax = axs[0]
    levels = np.linspace(phat.min(), phat.max(), 20)
    im = ax.contourf(X, Y, phat, levels=levels, cmap=cmap)
    ax.set_xlim(lower_lim, upper_lim)
    ax.set_ylim(lower_lim, upper_lim)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Density from KDE (BW=%.2f)' % bw)

    ax = axs[1]
    ax.scatter(data[:,0], data[:,1], s=35, c=color, linewidth=0, alpha=0.6)
    ax.set_xlabel('X')
    plt.show()

In [ ]:
# Plot the same thing for the other two data
run_kde_and_plot(m_data2, bw=selected_bw, color='red', cmap='Reds')
run_kde_and_plot(m_data5, bw=selected_bw, color='green', cmap='Greens')

## 4. Automated Bandwidth Selection

Cross-validation can be used to select the bandwidth automatically. Cross-validation is a model validation technique for assessing how the results will generalize to an independent data set. In K-fold cross-validation, randomized data are splitted into K sets, and K-1 sets are used for estimating the density (train set) and 1 set is used for evaluation (validation set). We do this for K times, and score is calculated for the validation set at each run. The overall cross-validation score is the mean of the M scores. (More info: [Wikipedia: Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_%28statistics%29))

Scikit-learn has a nice package called **`GridSearchCV`** that does all the job for us! It uses **`score()`** function in the object to calculate the score. **`KernelDensity`** class has a function **`score(valX)`** that returns the total log probability of the validation data **`valX`** under the model. **`GridSearchCV`** will calculate the cross-validation score for each bandwidth value, and then return the bandwidth that gave the highest score. <br/>(Package info: [sklearn.model_selection.GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

We will use 10-fold cross-validation.

In [ ]:
min_bw = 30
max_bw = 120
grid = GridSearchCV(KernelDensity(metric='euclidean', kernel='gaussian'),
                    {'bandwidth': np.linspace(min_bw, max_bw, 50)}, cv=10) # 10-fold cross-validation
grid.fit(m_data1)
print(grid.best_params_)
bw_cv = grid.best_params_['bandwidth'] # Bandwidth value saved in 'bw_cv'

In [ ]:
run_kde_and_plot(m_data1, bw=bw_cv, color='blue', cmap='Blues')

In [ ]:
grid.fit(m_data2)
print(grid.best_params_)
bw_cv = grid.best_params_['bandwidth'] # Bandwidth value saved in 'bw_cv'

run_kde_and_plot(m_data2, bw=bw_cv, color='red', cmap='Reds')

In [ ]:
grid.fit(m_data5)
print(grid.best_params_)
bw_cv = grid.best_params_['bandwidth'] # Bandwidth value saved in 'bw_cv'

run_kde_and_plot(m_data5, bw=bw_cv, color='green', cmap='Greens')

## 5. Difference between KDEs

In [ ]:
bw = 80

phat1 = run_kde(m_data1, bandwidth=bw, metric='euclidean', kernel='gaussian') 
phat2 = run_kde(m_data2, bandwidth=bw, metric='euclidean', kernel='gaussian') 
phat5 = run_kde(m_data5, bandwidth=bw, metric='euclidean', kernel='gaussian')

In [ ]:
# Print out the maximum value of each function
# This is to set up the range in the contour plot (plot them in the same scale)

print(np.max(phat1))
print(np.max(phat2))
print( np.max(phat5))

In [ ]:
levels = np.linspace(0, 7.5e-6, 20)

fig, axs = plt.subplots(1,3, figsize=(13,4), sharey=True)
ax = axs[0]
im = ax.contourf(X, Y, phat1, levels=levels, cmap='Greys')
ax.set_xlim(lower_lim, upper_lim)
ax.set_ylim(lower_lim, upper_lim)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('$\hat{p}_1$', fontsize=18)

ax = axs[1]
im = ax.contourf(X, Y, phat2, levels=levels, cmap='Greys')
ax.set_xlim(lower_lim, upper_lim)
ax.set_ylim(lower_lim, upper_lim)
ax.set_xlabel('X')
ax.set_title('$\hat{p}_2$', fontsize=18)

ax = axs[2]
im = ax.contourf(X, Y, phat5, levels=levels, cmap='Greys')
ax.set_xlim(lower_lim, upper_lim)
ax.set_ylim(lower_lim, upper_lim)
ax.set_xlabel('X')
ax.set_title('$\hat{p}_5$', fontsize=18)
plt.show()

In [ ]:
# Print out the minimum and maximum values of the functions
# This is to set up the range in the contour plot
# We can see the biggest difference between data2 and data5

print(np.min(phat1-phat2), np.max(phat1-phat2))
print(np.min(phat1-phat5), np.max(phat1-phat5))
print(np.min(phat2-phat5), np.max(phat2-phat5))

In [ ]:
# Normalize the probability values to percentage
max_val = np.max(phat2-phat5)

phat12 = (phat1-phat2) / max_val * 100.0
phat15 = (phat1-phat5) / max_val * 100.0
phat25 = (phat2-phat5) / max_val * 100.0

In [ ]:
levels = np.linspace(-100, 100, 20)

fig, axs = plt.subplots(1,4, figsize=(13,4), gridspec_kw = {'width_ratios':[5,5,5,1]})
ax = axs[0]
im = ax.contourf(X, Y, phat12, levels=levels, cmap='RdBu')
ax.set_xlim(lower_lim, upper_lim)
ax.set_ylim(lower_lim, upper_lim)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('$\hat{p_1} - \hat{p_2}$', fontsize=18)

ax = axs[1]
im = ax.contourf(X, Y, phat15, levels=levels, cmap='RdBu')
ax.set_xlim(lower_lim, upper_lim)
ax.set_ylim(lower_lim, upper_lim)
ax.set_xlabel('X')
ax.set_title('$\hat{p_1} - \hat{p_5}$', fontsize=18)

ax = axs[2]
im = ax.contourf(X, Y, phat25, levels=levels, cmap='RdBu')
ax.set_xlim(lower_lim, upper_lim)
ax.set_ylim(lower_lim, upper_lim)
ax.set_xlabel('X')
ax.set_title('$\hat{p_2} - \hat{p_5}$', fontsize=18)

fig.colorbar(im, cax=axs[3])
plt.show()